In [1]:
import pandas as pd
import geopandas as gpd
import geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt

import folium
from folium.plugins import FastMarkerCluster

## Geocode the addresses from Real Estate transactions database
Data taken from NYdatabase.com

In [7]:
test_locator = Nominatim(user_agent="my_Geocoder")
location = test_locator.geocode("911 East Shore Drive, Tompkins County, NY, USA")

In [9]:
print(location.address)
print("Latitude ={}, Longitude ={}".format(location.latitude, location.longitude))    # Test location Lat-Long checks out

911, East Shore Drive, Renwick, Ithaca Town, Tompkins County, New York, 14850, United States of America
Latitude =42.46469782009001, Longitude =-76.49928169371793
